In [1]:
import os,sys
import matplotlib.pyplot as plt
#import pandas as pd  # AC got a 'no module named pandas' error.  Do we need it?
import numpy as np
import math
import csv
import scipy

# Eventually we'll have to find a common pathway, for now just change if this is commented out.
sys.path.insert(0,'/Users/dtn1/gsas2full/GSASII/') # needed to "find" GSAS-II modules
#sys.path.insert(0,'/Users/creuzige/gsas2full/envs/gsas2pkg/GSASII/') # needed to "find" GSAS-II modules

import GSASIIscriptable as G2sc

Failed to run pyspg in C:\Users\dtn1\gsas2full\GSASII\bindist
error: No module named 'pyspg'
Failed to run pyspg in C:\Users\dtn1\gsas2full\GSASII\bindist
error: No module named 'pyspg'
Attempting to download GSAS-II binary files...
Running svn command:
  C:\Users\dtn1\gsas2full\Library\bin\svn.exe list https://subversion.xray.aps.anl.gov/pyGSAS/Binaries/ --non-interactive --trust-server-cert 
FYI: The current numpy version, 1.19.2, is newer than the newest dist version 1.19
Loading files to C:\Users\dtn1\gsas2full\GSASII\bindist\
  from https://subversion.xray.aps.anl.gov/pyGSAS/Binaries/win_64_p3.8_n1.19/

svn command:  C:\Users\dtn1\gsas2full\Library\bin\svn.exe switch https://subversion.xray.aps.anl.gov/pyGSAS/Binaries/win_64_p3.8_n1.19/ C:\Users\dtn1\gsas2full\GSASII\bindist\ --non-interactive --trust-server-cert --accept theirs-conflict --force -rHEAD --ignore-ancestry 

=== Output from svn switch===========================================
D    C:\Users\dtn1\gsas2full\GSASII\bind

In [2]:
def LambdaAng2keV(Lambda):
    '''
    Convert between wavelength in angstroms and energy in keV
    '''
    keV=scipy.constants.h*scipy.constants.c/(Lambda*1e-10*scipy.constants.value("electron volt")*1000)
    return keV

In [3]:
# Theta from Ferrite 110, from Jatczak
Theta= 44.60/2 # degrees
# Theta= 44.60/2 # degrees
print(Theta)


# Cu radiation from Jatczak
Lambda=1.54178 # angstrom

# Cu radiation from GSASII
#Lambda=1.5405 # angstrom, 8.0477 keV

# K-beta peak
#Lambda=1.392 # angstrom, 8.0477 keV
print(Lambda)

22.3
1.54178


In [4]:
phase_dict = {
    'Fe': {
        "atom_site_type_symbol": "Fe",# CIF term
        "atom_site_occupancy":0.8, # CIF term
        "atom_site_occupancy_uncert":0.08 # estimage
    },
    'Ni': {
        "atom_site_type_symbol": "Ni",# CIF term
        "atom_site_occupancy":0.1, # CIF term
        "atom_site_occupancy_uncert":0.01 #      
    },
    'Cr': {
        "atom_site_type_symbol": "Cr",# CIF term
        "atom_site_occupancy": 0.1, # CIF term
        "atom_site_occupancy_uncert":0.01 #  
    }
}

In [27]:
def compute_structure_factor(element,theta,llambda,phase_dict,method):
    
    sTL=math.sin(math.radians(theta))/llambda

    key = element
    
    # Find the coefficients to calculate scattering factor
    FormFactor=G2sc.G2elem.GetFormFactorCoeff(phase_dict[key]["atom_site_type_symbol"])[0]
    #print(FormFactor)
    # Use coefficients to calculate the first term scattering
    ScatteringFactor0=(G2sc.G2elem.ScatFac(FormFactor, (sTL**2))[0])
    #print(ScatteringFactor0)

    # Find the electron orbital structures
    Orbitals=G2sc.G2elem.GetXsectionCoeff(phase_dict[key]["atom_site_type_symbol"])
    (f_prime, f_2prime, mu)= G2sc.G2elem.FPcalc(Orbitals,LambdaAng2keV(llambda))
    #print(f_prime)
    #print(Phase[key]["atom_site_occupancy"])
    
    if method == 'formula':
        mean_scattering = (ScatteringFactor0+f_prime)*phase_dict[key]["atom_site_occupancy"]
        sd_scattering = (ScatteringFactor0+f_prime)*phase_dict[key]["atom_site_occupancy_uncert"]
    
    elif method == 'mcmc':
        n = 20000
        mean_scattering = mean_scattering = (ScatteringFactor0+f_prime)*phase_dict[key]["atom_site_occupancy"]
        loc = phase_dict[key]["atom_site_occupancy"]
        scale = phase_dict[key]["atom_site_occupancy_uncert"]
        sd_scattering = np.std( (ScatteringFactor0+f_prime)*np.random.normal(loc,scale,n) )
      
    return mean_scattering, sd_scattering
    

In [28]:
print(compute_structure_factor(element='Fe',theta=44.60/2,llambda=1.54178,phase_dict=phase_dict,method='formula'))
print(compute_structure_factor(element='Ni',theta=44.60/2,llambda=1.54178,phase_dict=phase_dict,method='formula'))
print(compute_structure_factor(element='Cr',theta=44.60/2,llambda=1.54178,phase_dict=phase_dict,method='formula'))

(13.879738329028136, 1.3879738329028135)
(1.7548373453133548, 0.17548373453133548)
(1.6527916752151481, 0.1652791675215148)


In [30]:
print(compute_structure_factor(element='Fe',theta=44.60/2,llambda=1.54178,phase_dict=phase_dict,method='mcmc'))
print(compute_structure_factor(element='Ni',theta=44.60/2,llambda=1.54178,phase_dict=phase_dict,method='mcmc'))
print(compute_structure_factor(element='Cr',theta=44.60/2,llambda=1.54178,phase_dict=phase_dict,method='mcmc'))

(13.879738329028136, 1.3915583809854073)
(1.7548373453133548, 0.1756796610483934)
(1.6527916752151481, 0.16648079198163457)
